# Data preprocessing
### Input:

In [ ]:
import os
import sys

def find_import(path='.', prevpath=None):
    if path == prevpath:
        return ''
    path = os.path.abspath(path)
    cfgfile = os.path.join(path, 'tokens.py')
    if os.path.exists(cfgfile):
        return cfgfile
    return find_import(os.path.dirname(path), path)

module_path = find_import()
projdir = os.path.dirname(module_path)
if projdir not in sys.path:
    sys.path.append(projdir)

from tokens import sql_config 

host = sql_config['host']
database='roomekbot$offers'
user = sql_config['user']
password = sql_config['password']
sql_query = "select * from offers where city = 'Warszawa' and business_type = 'rent'"

### Import database

In [ ]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host=host, database=database, user=user, password=password)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    records = cursor.fetchall()
    
    sql_query = "describe offers"
    cursor = connection.cursor()
    cursor.execute(sql_query)
    titles = cursor.fetchall()
    titles = [x[0] for x in titles]
    
except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()

### Create dataframe

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data=records, columns=titles)
df.head()

### Get weights from SQL

In [ ]:
try:
    connection = mysql.connector.connect(host=host, database=database, user=user, password=password)
    cursor = connection.cursor(buffered=True, dictionary=True)
    sql_query = "select * from weights;"
    cursor.execute(sql_query)
    weights = cursor.fetchone()
    weights = pd.DataFrame.from_dict(weights, orient='index')
    
except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()
        
weights.drop(weights.tail(2).index,inplace=True) #drops modification_time and creation_time rows
weights.columns = ['Weight']
weights

### Statistics - show amount of empty cells

In [ ]:
# round(df.isnull().sum()/len(df)*1000)/10

### Replace None and Nan with mean value or 0

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
# specify which columns to replace with mean value...
# imputer = imputer.fit(df[:, 1:3])
# df[:, 1:3] = imputer.transform(df[:, 1:3])

# ... and which replace with zero:
# df[10:14] = df.fillna(0, inplace=True)

# or replace all with zeros:
df.fillna(0, inplace=True)
# df.head()

### Select only valuable data

In [ ]:
X = df.drop([
    'price',
    'parsed_fields',
    'offer_from',
    'offer_url',
    'offer_name',
    'offer_thumbnail_url',
    'offer_id',
    'offer_text',
    'creation_time',
    'modification_time',
    'city',
    'housing_type',
    'business_type',
    'pet_friendly',
    'rental_for_students',
    'non_smokers_only',
    'street'], axis=1)

X_text = X[['district',
            'apartment_level',
            'type_of_building',
            'type_of_market',
            'building_material',
            'windows',
            'heating',
            'type_of_ownership',
            'type_of_room',
            'preferred_locator',
            'fit_out']]

X_other = X[['location_latitude',
             'location_longitude',
#              'date_of_the_offer',
             'ready_from',
             'scraped_ranking',]]

X_irrelevant = X[['additional_rent',
             'building_year',
             'internet',
             'cable_tv',
             'bathtub',
             'utility_room',
             'terrace',
             'balcony',
             'separate_kitchen',
             'basement',
             'virtual_walk',
             'two_level_apartment',
             'connecting_room',
             'closed_terrain',
             'monitoring_or_security',
             'entry_phone',
             'antibulglar_doors_windows',
             'antibulglar_blinds',
             'alarm_system',
             'tv',
             'phone',
             'AC',
             'garden',
             'oven',
             'fridge',
             'cooker',
             'price_per_m2',
             'dishwasher',
             'washing_machine',
             'elevator',
             'security_deposit',]]

X = X.drop(list(X_text.columns)+list(X_other.columns)+list(X_irrelevant.columns), axis=1)
X.head()

### Scale

In [ ]:
now = pd.Timestamp.now()
diff = (now - df['date_of_the_offer'])/np.timedelta64(1,'h')
X['date_of_the_offer'] = diff
min = X['date_of_the_offer'].min()
max = X['date_of_the_offer'].max()
X['date_of_the_offer'] = 1/(X['date_of_the_offer']/min)**4

In [ ]:
# TODO NAPRAW SKALOWANIE WSZYZTKICH PÓL!

# def scale_uni(n):
#     return n.min()/n

# d = scale_uni(X['total_price'])
# X['total_price'] #.min()


### Evaluate scores

In [ ]:
common_columns = list(X.columns)
results = np.multiply(X, weights.T[common_columns])
row_sum = np.sum(results, axis=1)
results['score'] = row_sum

### Sort by best score

In [ ]:
results['offer_url'] = df['offer_url']
results = results.sort_values(by=['score'], ascending=False)

results.head()

### Eliminate Outliers

In [ ]:
def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [ ]:
def remove_outliers(x,y):
    xf = x[~np.logical_or(is_outlier(x), is_outlier(y))]
    yf = y[~np.logical_or(is_outlier(x), is_outlier(y))]
    return xf, yf

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 3), dpi= 80, facecolor='w', edgecolor='b')

plt.subplot(121)
x1=X['amount_of_rooms']
y1=X['total_price']
x1, y1 = remove_outliers(x1,y1)
plt.title("amount_of_rooms")
plt.xlabel('total_price')
plt.ylabel('total_price')
# calc the trendline
p1 = np.poly1d(np.polyfit(x1, y1, 3))
t1 = np.linspace(np.amin(x1), np.amax(x1), 200)
plt.plot(x1, y1, '.', t1, p1(t1), '-')

plt.subplot(122)
x2=X['area']
y2=X['total_price']
x2, y2 = remove_outliers(x2,y2)
plt.title("Area vs price")
plt.xlabel('area')
plt.ylabel('price')
p2 = np.poly1d(np.polyfit(x2, y2, 3))
t2 = np.linspace(np.amin(x2), np.amax(x2), 200)
plt.plot(x2, y2, '.', t2, p2(t2), '-')

plt.show()

In [ ]:
# import seaborn as sns

# sns.set(style="ticks", color_codes=True)
# iris = sns.load_dataset("iris")
# g = sns.pairplot(iris)

# df = pd.DataFrame(dict(time=np.arange(500),
#                        value=np.random.randn(500).cumsum()))
# g = sns.relplot(x="time", y="value", kind="line", data=df)
# g.fig.autofmt_xdate()

# plt.show()

# sns.lmplot("equipment", "price", data=filtered_class, hue="gears", fit_reg=False, col='modelLine', col_wrap=2)


### Feature Scaling

In [ ]:
# TODO IMPROVE SCALING! 
# obecnie jest blad ze nie zwraca w zakresie o-1 tylko np. -5.5-0.2

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_std=True)

X_numeric_sc = scaler.fit_transform(X_numeric)
temp = pd.DataFrame(data = X_numeric_sc, columns = X_numeric.columns)
temp.head()

### Encode labels for categorical data - text into numbers

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()

X[:,0] = labelencoder_X.fit_transform(X[:,0])

# enc = OneHotEncoder(categorical_features = [0])
# X = enc.fit_transform(X).toarray()

### Dispatch dependent matrix and independent vector

In [ ]:
# # https://towardsdatascience.com/the-complete-beginners-guide-to-data-cleaning-and-preprocessing-2070b7d4c6d

# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# # from sklearn.cross_validation import train_test_split

# # Matrix of dependent variables:
# X = df.drop(['price', 'parsed_fields', 'offer_from', 'offer_url', 'offer_name', 'offer_thumbnail_url', 'offer_id', 'offer_text', 'creation_time', 'modification_time', 'street'], axis=1)
# X.head()
# # X=X.values

# # #vector of independent variables:
# # y = df.loc[:, ['price']]
# # y=y.values
# # X

### Split dataset into train set and test set

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)